# Homework 3: Relation extraction using distant supervision

In [1]:
__author__ = "Bill MacCartney"
__version__ = "CS224U, Stanford, Spring 2019"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baseline](#Baseline)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 point]](#Different-model-factory-[1-point])
  1. [Directional unigram features [2 points]](#Directional-unigram-features-[2-points])
  1. [The part-of-speech tags of the "middle" words [2 points]](#The-part-of-speech-tags-of-the-"middle"-words-[2-points])
  1. [Your original system [4 points]](#Your-original-system-[4-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to the developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [2]:
import os
import rel_ext
from sklearn.linear_model import LogisticRegression

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [3]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [4]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [5]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [6]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [7]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [8]:
splits

{'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

## Baseline

In [9]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [10]:
featurizers = [simple_bag_of_words_featurizer]

In [11]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [12]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.841      0.388      0.682        340       5716
author                    0.786      0.562      0.728        509       5885
capital                   0.621      0.189      0.427         95       5471
contains                  0.807      0.585      0.750       3904       9280
film_performance          0.792      0.563      0.732        766       6142
founders                  0.803      0.387      0.661        380       5756
genre                     0.490      0.141      0.328        170       5546
has_sibling               0.828      0.251      0.567        499       5875
has_spouse                0.874      0.327      0.655        594       5970
is_a                      0.703      0.223      0.492        497       5873
nationality               0.653      0.156      0.399        301       5677
parents     

Studying model weights might yield insights:

In [13]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.485 Córdoba
     2.462 Valais
     2.429 Taluks
     ..... .....
    -1.404 century
    -1.600 thereby
    -2.003 America

Highest and lowest feature weights for relation author:

     3.152 author
     2.683 book
     2.599 novel
     ..... .....
    -2.004 superhero
    -2.066 or
    -4.572 17th

Highest and lowest feature weights for relation capital:

     3.361 capital
     1.913 city
     1.798 km
     ..... .....
    -1.192 and
    -1.302 borough
    -1.424 ’

Highest and lowest feature weights for relation contains:

     2.359 bordered
     2.144 districts
     2.061 neighborhood
     ..... .....
    -3.529 6th
    -3.837 Ceylon
    -6.036 Bronx

Highest and lowest feature weights for relation film_performance:

     4.147 starring
     3.835 co-starring
     3.345 alongside
     ..... .....
    -1.928 Mappillai
    -1.936 series
    -1.996 Westminster

Highest and lowest feature weights for relation founders:

 

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 point]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A call to `rel_ext.experiment` training on the 'train' part of `splits` and assessing on its `dev` part, with `featurizers` as defined above in this notebook and the `model_factory` set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values.

In [14]:
from sklearn.svm import *

model_factory = lambda: SVC(kernel='linear')
_ = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.775      0.365      0.633        340       5716
author                    0.725      0.611      0.699        509       5885
capital                   0.692      0.284      0.538         95       5471
contains                  0.784      0.596      0.738       3904       9280
film_performance          0.760      0.624      0.728        766       6142
founders                  0.782      0.424      0.669        380       5756
genre                     0.551      0.224      0.426        170       5546
has_sibling               0.787      0.244      0.545        499       5875
has_spouse                0.806      0.350      0.640        594       5970
is_a                      0.612      0.264      0.484        497       5873
nationality               0.520      0.173      0.371        301       5677
parents     

### Directional unigram features [2 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example. The precise nature of the mark you add for the two cases doesn't make a difference to the model.

2. The macro-average F-score on the `dev` set that you obtain from running `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it on Piazza!)

In [15]:
# step 1: define directional_bag_of_words_featurizer
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[(word, "forward")] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[(word, "backward")] += 1
    return feature_counter
directional_featurizers = [directional_bag_of_words_featurizer]





# step 2: get the macro-average F score using all default parameters+functions
p2_result = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=directional_featurizers,
    model_factory=lambda: LogisticRegression(fit_intercept=True, solver='liblinear'),
    verbose=True
)
print("The macro-average F-score is", 0.605)


# step 3: # of feature names does the vectorizer have for the experiment run in the previous step
print("number of feature names from the previous steps", len(p2_result['vectorizer'].get_feature_names()))





relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.857      0.424      0.711        340       5716
author                    0.832      0.595      0.771        509       5885
capital                   0.679      0.200      0.459         95       5471
contains                  0.818      0.663      0.782       3904       9280
film_performance          0.839      0.658      0.795        766       6142
founders                  0.807      0.397      0.669        380       5756
genre                     0.759      0.259      0.547        170       5546
has_sibling               0.831      0.246      0.564        499       5875
has_spouse                0.869      0.345      0.666        594       5970
is_a                      0.759      0.241      0.531        497       5873
nationality               0.624      0.209      0.447        301       5677
parents     

### The part-of-speech tags of the "middle" words [2 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Don't forget the start and end tags, to model those environments properly!

2. The macro-average F-score on the `dev` set that you obtain from running `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `experiment` as exemplified above in this notebook.)

Note: To parse `middle_POS`, one splits on whitespace to get the `word/TAG` pairs. Each of these pairs `s` can be parsed with `s.rsplit('/', 1)`.

In [16]:
import nltk
# helper function
def create_bigram(ex):
    POS = []
    for word in ex.middle_POS.split(' '):
        if not word:
#             print("none")
            continue
        term, tag = word.rsplit('/', 1)
        POS.append(tag)
    # add </s> and <s> at the beginning of the first word and the end of the last word
    POS.append('</s>')
    POS.insert(0,'<s>')
    # convert POS into a bigram:
    bigrams = list(nltk.bigrams(POS))    
    return bigrams

# step 1: define middle_bigram_pos_tag_featurizer
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        # parse the middle_POS and get the 
        bigrams = create_bigram(ex)
        bigram_list = []
        for i in bigrams:
            i = " ".join(i)
            #bigram_list.append(i)
            feature_counter[i] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        # parse the middle_POS and get the 
        bigrams = create_bigram(ex)
        bigram_list = []
        for i in bigrams:
            i = " ".join(i)
            #bigram_list.append(i)
            feature_counter[i] += 1
    return feature_counter

In [17]:
# experiment 
middle_featurizers = [middle_bigram_pos_tag_featurizer]
p3_result = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers= middle_featurizers,
    model_factory=lambda: LogisticRegression(fit_intercept=True, solver='liblinear'),
    verbose=True
)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.858      0.338      0.656        340       5716
author                    0.730      0.334      0.590        509       5885
capital                   0.586      0.179      0.403         95       5471
contains                  0.757      0.591      0.717       3904       9280
film_performance          0.707      0.444      0.632        766       6142
founders                  0.560      0.171      0.385        380       5756
genre                     0.500      0.182      0.371        170       5546
has_sibling               0.724      0.168      0.436        499       5875
has_spouse                0.780      0.269      0.566        594       5970
is_a                      0.542      0.167      0.374        497       5873
nationality               0.377      0.066      0.195        301       5677
parents     

In [18]:
print("macro-average F-score is:", 0.443)

macro-average F-score is: 0.443


### Your original system [4 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?
- Consider adding features based on WordNet synsets. Here's a little code to get you started with that:
  ```
  from nltk.corpus import wordnet as wn
  dog_compatible_synsets = wn.synsets('dog', pos='n')
 ```

In [33]:
# import packages
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
# Try out different classifier models, from sklearn and elsewhere
classifiers = [
    KNeighborsClassifier(3),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
   AdaBoostClassifier(),
    ]

In [25]:
# decision tree with max_depth = 5
DecisionTreeClassifier_result = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=lambda: DecisionTreeClassifier(max_depth=5),
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.871      0.338      0.662        340       5716
author                    0.767      0.361      0.626        509       5885
capital                   0.735      0.263      0.541         95       5471
contains                  0.799      0.479      0.705       3904       9280
film_performance          0.773      0.454      0.678        766       6142
founders                  0.931      0.284      0.640        380       5756
genre                     0.583      0.082      0.263        170       5546
has_sibling               0.799      0.230      0.535        499       5875
has_spouse                0.806      0.335      0.629        594       5970
is_a                      0.730      0.163      0.430        497       5873
nationality               0.840      0.070      0.262        301       5677
parents     

In [30]:
# RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
RandomForest_result = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=lambda: RandomForestClassifier(max_depth=5, n_estimators=15),
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   1.000      0.035      0.155        340       5716
author                    0.000      0.000      0.000        509       5885
capital                   0.000      0.000      0.000         95       5471
contains                  0.919      0.070      0.267       3904       9280
film_performance          0.000      0.000      0.000        766       6142
founders                  0.000      0.000      0.000        380       5756
genre                     0.000      0.000      0.000        170       5546
has_sibling               0.000      0.000      0.000        499       5875
has_spouse                0.000      0.000      0.000        594       5970
is_a                      0.000      0.000      0.000        497       5873
nationality               0.000      0.000      0.000        301       5677
parents     

/Users/yatong_chen/anaconda3/envs/nlu/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
 # MLPClassifier(alpha=1)
MLP_result = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=lambda: MLPClassifier(alpha=1),
    verbose=True)

/Users/yatong_chen/anaconda3/envs/nlu/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/Users/yatong_chen/anaconda3/envs/nlu/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [ ]:
# AdaBoostClassifier()
AdaBoost_result = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=lambda: AdaBoostClassifier(),
    verbose=True)

## Bake-off [1 point]

For the bake-off, we will release a test set right after class on April 29. The announcement will go out on Piazza. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

To enter the bake-off, upload this notebook on Canvas:

https://canvas.stanford.edu/courses/99711/assignments/187248

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

The bake-off will close at 4:30 pm on May 1. Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

In [20]:
# Enter your bake-off assessment code in this cell. 
# Please do not remove this comment.


In [21]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported 
# by the code above. Please enter only a number between 
# 0 and 1 inclusive. Please do not remove this comment.
